12/17/18
BTC Prediction

In [1]:
import requests
import io
import pandas as pd

In [2]:
# 1Token's API address
tokenapi = 'http://hist-quote.1tokentrade.cn'

In [3]:
# Retrieve API token
token_txt_path = '/Users/jonathancai/github/btc-analysis/token.txt'
with open(token_txt_path) as file:
    ot_key = file.read().splitlines()[0]

In [4]:
# Query all contracts for a certain date
r = requests.get(tokenapi + '/ticks/contracts?date=2018-02-02', headers = {'ot-key' : ot_key})
# r.json()

['allcoin/a0101.cnet',
 'allcoin/acc.btc',
 'allcoin/act.btc',
 'allcoin/act.cnet',
 'allcoin/aic.cnet',
 'allcoin/aicc.btc',
 'allcoin/aicc.ckusd',
 'allcoin/aidoc.ckusd',
 'allcoin/aidoc.eth',
 'allcoin/atn.btc',
 'allcoin/atn.eth',
 'allcoin/atn.qtum',
 'allcoin/awr.ckusd',
 'allcoin/awr.qtum',
 'allcoin/bash.btc',
 'allcoin/bcd.btc',
 'allcoin/bcd.ckusd',
 'allcoin/bch.btc',
 'allcoin/bch.ckusd',
 'allcoin/bch.cnet',
 'allcoin/bec.ckusd',
 'allcoin/bot.qtum',
 'allcoin/btc.ckusd',
 'allcoin/bte.cnet',
 'allcoin/btf.ckusd',
 'allcoin/btg.btc',
 'allcoin/btg.ckusd',
 'allcoin/btg.cnet',
 'allcoin/btm.btc',
 'allcoin/btm.eth',
 'allcoin/cfs.btc',
 'allcoin/cfun.qtum',
 'allcoin/cnet.ckusd',
 'allcoin/dbc.ckusd',
 'allcoin/dbc.eth',
 'allcoin/dwc.cnet',
 'allcoin/ent.qtum',
 'allcoin/eth.btc',
 'allcoin/eth.ckusd',
 'allcoin/fid.btc',
 'allcoin/game.btc',
 'allcoin/gnx.ckusd',
 'allcoin/gnx.eth',
 'allcoin/gp.ckusd',
 'allcoin/gxs.btc',
 'allcoin/hlc.ckusd',
 'allcoin/hlc.qtum',
 'allc

In [5]:
# Query a single price at date/contract
params = {
    'contract' : 'binance/btc.usdt',
    'date' : '2018-02-02',
}
r = requests.get(tokenapi + '/ticks/simple' , headers = {'ot-key' : ot_key}, params = params)
# r

<Response [200]>

In [17]:
# Query "candlestick" data
params = {
#     'since' : '2013-12-10',
#     'since' : '2018-01-01',
    'since' : '2018-12-20',
    'until' : '2018-12-21',
    'contract' : 'binance/btc.usdt',
    'duration' : '1d',
    'format' : 'csv'
}
r = requests.get(tokenapi + '/candles' , headers = {'ot-key' : ot_key}, params = params, stream = True)

# Write to local file
btc_data_path = '/Users/jonathancai/github/btc-analysis/btc-historical-data.csv'
df = pd.read_csv(io.StringIO(r.text))   
df.to_csv(btc_data_path)
print("Number of rows:", df.size / 6)
df.head()

Number of rows: 1.0


,open,close,high,low,volume,timestamp
0,3660.01,4049.62,4120.0,3635.0,101543.835986,1545264000
